<a href="https://colab.research.google.com/github/kr1347/GAN-Model/blob/main/GAN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')











Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch torchvision matplotlib


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hearfool/vggface2")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/hearfool/vggface2/versions/1


#Part 2: Training Setup

This includes:

Dataloader
Optimizers
Loss function
Fixed noise for visualization
Checkpoint directory setup

In [ ]:
import os
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image

input_dir = "/root/.cache/kagglehub/datasets/hearfool/vggface2/versions/1"
output_dir = "./vggface2_processed"
os.makedirs(output_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

count = 0
max_images = 25000  # Limit to speed up training

for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".jpg"):
            try:
                path = os.path.join(root, file)
                img = Image.open(path).convert("RGB")
                img = transform(img)

                # Extract class name from the directory structure
                class_name = os.path.basename(os.path.dirname(path))

                # Create class subdirectory if it doesn't exist
                class_dir = os.path.join(output_dir, class_name)
                os.makedirs(class_dir, exist_ok=True)

                save_image(img, os.path.join(class_dir, f"{count}.png"))
                count += 1
                if count >= max_images:
                    break
            except Exception as e:
                print(f"Error: {e}")
    if count >= max_images:
        break

print(f"Saved {count} preprocessed images.")

Saved 25000 preprocessed images.


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

image_size = 128
batch_size = 128
z_dim = 100

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)  # Normalize to [-1, 1]
])

dataset = ImageFolder(root="./vggface2_processed", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

gen = Generator(z_dim=z_dim).to(device)
disc = Discriminator().to(device)
opt_gen = torch.optim.Adam(gen.parameters(), lr=1e-4, betas=(0.5, 0.999))

opt_disc = torch.optim.Adam(disc.parameters(), lr=1e-4, betas=(0.5, 0.999))

criterion = nn.BCELoss()

fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)


In [ ]:
import os

checkpoint_dir = "/content/drive/MyDrive/dcgan_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

sample_dir = "/content/drive/MyDrive/dcgan_samples"
os.makedirs(sample_dir, exist_ok=True)


#Model defined


In [ ]:
import torch.nn as nn
from torch.nn.utils import spectral_norm

class Generator(nn.Module):
    def __init__(self, z_dim=100, channels_img=3, features_g=32):
        super().__init__()
        self.net = nn.Sequential(
            self._block(z_dim, features_g * 8, 4, 1, 0),
            self._block(features_g * 8, features_g * 4, 4, 2, 1),
            self._block(features_g * 4, features_g * 2, 4, 2, 1),
            self._block(features_g * 2, features_g, 4, 2, 1),
            nn.ConvTranspose2d(features_g, channels_img, 4, 2, 1),
            nn.Tanh()
        )

    def _block(self, in_channels, out_channels, k, s, p):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, k, s, p, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True)
        )

    def forward(self, x):
        return self.net(x)

class Discriminator(nn.Module):
    def __init__(self, channels_img=3, features_d=32):
        super().__init__()
        self.net = nn.Sequential(
            spectral_norm(nn.Conv2d(channels_img, features_d, 4, 2, 1)),
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            spectral_norm(nn.Conv2d(features_d * 8, 1, 4, 1, 0)),
            nn.Sigmoid()
        )

    def _block(self, in_channels, out_channels, k, s, p):
        return nn.Sequential(
            spectral_norm(nn.Conv2d(in_channels, out_channels, k, s, p, bias=False)),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.net(x)


Training Setup

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

image_size = 64
batch_size = 128
z_dim = 100

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = ImageFolder(root="./vggface2_processed", transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

device = "cuda" if torch.cuda.is_available() else "cpu"

gen = Generator(z_dim=z_dim).to(device)
disc = Discriminator().to(device)

opt_gen = torch.optim.Adam(gen.parameters(), lr=1e-4, betas=(0.5, 0.999))
opt_disc = torch.optim.Adam(disc.parameters(), lr=1e-4, betas=(0.5, 0.999))

criterion = nn.BCELoss()
fixed_noise = torch.randn(64, z_dim, 1, 1).to(device)

checkpoint_dir = "/content/drive/MyDrive/dcgan_checkpoints"
sample_dir = "/content/drive/MyDrive/dcgan_samples"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(sample_dir, exist_ok=True)


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

def show_generated_image(image_path, epoch):
    img = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Generated Samples - Epoch {epoch}")
    plt.show()

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

gen.apply(weights_init)
disc.apply(weights_init)

num_epochs = 100
start_epoch = 0
best_lossG = float("inf")

for epoch in range(start_epoch, num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        real_noisy = real + 0.1 * torch.randn_like(real)
        fake_noisy = fake.detach() + 0.1 * torch.randn_like(fake)

        disc_real = disc(real_noisy).reshape(-1)
        disc_fake = disc(fake_noisy).reshape(-1)

        real_labels = torch.full_like(disc_real, 0.9)
        fake_labels = torch.zeros_like(disc_fake)

        lossD_real = criterion(disc_real, real_labels)
        lossD_fake = criterion(disc_fake, fake_labels)
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake).reshape(-1)
        gen_labels = torch.ones_like(output)
        lossG = criterion(output, gen_labels)

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}/{num_epochs}] [Batch {batch_idx}] "
                  f"Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

    print(f"[Epoch {epoch+1}/{num_epochs}] Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    sample_path = f"{sample_dir}/epoch_{epoch+1}.png"
    with torch.no_grad():
        fake = gen(fixed_noise)
        save_image(fake * 0.5 + 0.5, sample_path)

    show_generated_image(sample_path, epoch + 1)

    if lossG.item() < best_lossG:
        best_lossG = lossG.item()
        torch.save(gen.state_dict(), f"{checkpoint_dir}/best_generator.pth")
        print(f" Best generator saved at epoch {epoch+1} with Loss G: {best_lossG:.4f}")

    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'gen_state_dict': gen.state_dict(),
            'disc_state_dict': disc.state_dict(),
            'opt_gen_state_dict': opt_gen.state_dict(),
            'opt_disc_state_dict': opt_disc.state_dict(),
        }, f"{checkpoint_dir}/dcgan_epoch_{epoch+1}.pth")


In [ ]:
checkpoint_path = "/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_100.pth"  # Example path
resume_training = True  # Set to False if starting from scratch

if resume_training and os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}...")
    checkpoint = torch.load(checkpoint_path, map_location=device)

    gen.load_state_dict(checkpoint['gen_state_dict'])
    disc.load_state_dict(checkpoint['disc_state_dict'])
    opt_gen.load_state_dict(checkpoint['opt_gen_state_dict'])
    opt_disc.load_state_dict(checkpoint['opt_disc_state_dict'])

    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Starting from scratch...")
    start_epoch = 0


Loading checkpoint from /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_100.pth...
Resumed from epoch 100


In [ ]:
import os
import torch
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image

checkpoint_path = "/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_100.pth"
num_epochs = 50
z_dim = 100
best_lossG = float("inf")

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    gen.load_state_dict(checkpoint['gen_state_dict'])
    disc.load_state_dict(checkpoint['disc_state_dict'])
    opt_gen.load_state_dict(checkpoint['opt_gen_state_dict'])
    opt_disc.load_state_dict(checkpoint['opt_disc_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Checkpoint not found. Starting from scratch.")
    start_epoch = 0

def show_generated_image(image_path, epoch):
    img = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Generated Samples - Epoch {epoch}")
    plt.show()

for epoch in range(start_epoch, start_epoch + num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        real_noisy = real + 0.1 * torch.randn_like(real)
        fake_noisy = fake.detach() + 0.1 * torch.randn_like(fake)

        disc_real = disc(real_noisy).reshape(-1)
        disc_fake = disc(fake_noisy).reshape(-1)
        real_labels = torch.full_like(disc_real, 0.9)
        fake_labels = torch.zeros_like(disc_fake)

        lossD_real = criterion(disc_real, real_labels)
        lossD_fake = criterion(disc_fake, fake_labels)
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake).reshape(-1)
        gen_labels = torch.ones_like(output)
        lossG = criterion(output, gen_labels)

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}] [Batch {batch_idx}] Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

    print(f" [Epoch {epoch+1}] Final Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    sample_path = f"{sample_dir}/epoch_{epoch+1}.png"
    with torch.no_grad():
        fake = gen(fixed_noise)
        save_image(fake * 0.5 + 0.5, sample_path)
    show_generated_image(sample_path, epoch + 1)

    if lossG.item() < best_lossG:
        best_lossG = lossG.item()
        torch.save(gen.state_dict(), f"{checkpoint_dir}/best_generator.pth")
        print(f" Best generator saved at epoch {epoch+1} with Loss G: {best_lossG:.4f}")

    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'gen_state_dict': gen.state_dict(),
            'disc_state_dict': disc.state_dict(),
            'opt_gen_state_dict': opt_gen.state_dict(),
            'opt_disc_state_dict': opt_disc.state_dict(),
        }, f"{checkpoint_dir}/dcgan_epoch_{epoch+1}.pth")


In [ ]:
!pip install pytorch-fid


In [ ]:
import os
import shutil
from torchvision.utils import save_image
from glob import glob

!mkdir -p fid_real fid_fake

In [ ]:
real_images = glob('./vggface2_processed/*/*.png')[:1000]  # or change to 500 for speed
for i, path in enumerate(real_images):
    shutil.copy(path, f'fid_real/{i}.png')


In [ ]:
gen.load_state_dict(torch.load(f"{checkpoint_dir}/best_generator.pth"))
gen.eval()

with torch.no_grad():
    for i in range(1000):
        noise = torch.randn(1, z_dim, 1, 1).to(device)
        fake = gen(noise)
        save_image(fake * 0.5 + 0.5, f"fid_fake/{i}.png")


In [ ]:
!python -m pytorch_fid fid_real fid_fake


Downloading: "https://github.com/mseitzer/pytorch-fid/releases/download/fid_weights/pt_inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/pt_inception-2015-12-05-6726825d.pth
100% 91.2M/91.2M [00:00<00:00, 285MB/s]
100% 20/20 [00:01<00:00, 17.84it/s]
100% 20/20 [00:00<00:00, 28.82it/s]
FID:  88.12661568589647


In [ ]:
import os
import torch
from torchvision.utils import save_image
import shutil
from pytorch_fid.fid_score import calculate_fid_given_paths
from glob import glob

# === Paths ===
checkpoint_dir = "/content/drive/MyDrive/dcgan_checkpoints"
sample_dir = "/content/fid_temp/generated"
real_dir = "/content/fid_temp/real"
os.makedirs(sample_dir, exist_ok=True)
os.makedirs(real_dir, exist_ok=True)

# === Settings ===
z_dim = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

# === Load Real Images (first time only) ===
from glob import glob
from PIL import Image
from torchvision import transforms

real_images = glob('./vggface2_processed/*/*.png')[:1000]
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
])

# Populate real_dir (only once)
if len(os.listdir(real_dir)) < 1000:
    for i, path in enumerate(real_images):
        img = Image.open(path).convert("RGB")
        img = transform(img)
        save_image(img, f"{real_dir}/real_{i}.png")

# === Evaluate FID for Models ===
fid_scores = {}

for epoch in range(100, 150):  # Update as needed
    try:
        ckpt_path = f"{checkpoint_dir}/dcgan_epoch_{epoch}.pth"
        print(f"\n Evaluating model: {ckpt_path}")

        # Load Generator
        checkpoint = torch.load(ckpt_path, map_location=device)
        gen.load_state_dict(checkpoint['gen_state_dict'])
        gen.eval()

        # Clean generated folder
        shutil.rmtree(sample_dir)
        os.makedirs(sample_dir, exist_ok=True)

        # Generate fake images
        with torch.no_grad():
            for i in range(1000):  # Use 1000 to match real samples
                noise = torch.randn(1, z_dim, 1, 1, device=device)
                fake_img = gen(noise).detach().cpu()
                save_image(fake_img * 0.5 + 0.5, f"{sample_dir}/fake_{i}.png")

        # Calculate FID
        fid = calculate_fid_given_paths([sample_dir, real_dir], batch_size=32, device=device, dims=2048)
        fid_scores[epoch] = fid
        print(f" FID for epoch {epoch}: {fid:.4f}")

    except Exception as e:
        print(f" Error at epoch {epoch}: {e}")

# === Show Best Result ===
if fid_scores:
    best_epoch = min(fid_scores, key=fid_scores.get)
    print("\n BEST MODEL:")
    print(f"Epoch {best_epoch} with FID: {fid_scores[best_epoch]:.4f}")

    # Save FID results
    with open("fid_results.txt", "w") as f:
        for epoch, fid in fid_scores.items():
            f.write(f"Epoch {epoch}: FID = {fid:.4f}\n")
        f.write(f"\nBest Epoch: {best_epoch} | FID = {fid_scores[best_epoch]:.4f}\n")
else:
    print(" No FID scores were computed.")



 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_100.pth


100%|██████████| 32/32 [00:00<00:00, 46.04it/s]


 FID for epoch 100: 93.1466

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_101.pth
 Error at epoch 101: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_101.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_102.pth
 Error at epoch 102: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_102.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_103.pth
 Error at epoch 103: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_103.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_104.pth
 Error at epoch 104: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_104.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_105.pth


100%|██████████| 32/32 [00:00<00:00, 47.34it/s]


 FID for epoch 105: 87.7902

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_106.pth
 Error at epoch 106: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_106.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_107.pth
 Error at epoch 107: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_107.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_108.pth
 Error at epoch 108: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_108.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_109.pth
 Error at epoch 109: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_109.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_110.pth


...

 FID for epoch 135: 88.0372

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_136.pth
 Error at epoch 136: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_136.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_137.pth
 Error at epoch 137: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_137.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_138.pth
 Error at epoch 138: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_138.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_139.pth
 Error at epoch 139: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_139.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_140.pth


100%|██████████| 32/32 [00:00<00:00, 46.52it/s]


 FID for epoch 140: 88.4853

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_141.pth
 Error at epoch 141: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_141.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_142.pth
 Error at epoch 142: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_142.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_143.pth
 Error at epoch 143: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_143.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_144.pth
 Error at epoch 144: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_144.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_145.pth


100%|██████████| 32/32 [00:00<00:00, 46.80it/s]


 FID for epoch 145: 83.3406

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_146.pth
 Error at epoch 146: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_146.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_147.pth
 Error at epoch 147: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_147.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_148.pth
 Error at epoch 148: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_148.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_149.pth
 Error at epoch 149: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_149.pth'

 BEST MODEL:
Epoch 145 with FID: 83.3406


In [ ]:
import os
import torch
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image

checkpoint_path = "/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_145.pth"
num_epochs = 50
z_dim = 100
best_lossG = float("inf")

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    gen.load_state_dict(checkpoint['gen_state_dict'])
    disc.load_state_dict(checkpoint['disc_state_dict'])
    opt_gen.load_state_dict(checkpoint['opt_gen_state_dict'])
    opt_disc.load_state_dict(checkpoint['opt_disc_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Checkpoint not found. Starting from scratch.")
    start_epoch = 0

def show_generated_image(image_path, epoch):
    img = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Generated Samples - Epoch {epoch}")
    plt.show()

for epoch in range(start_epoch, start_epoch + num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        real_noisy = real + 0.1 * torch.randn_like(real)
        fake_noisy = fake.detach() + 0.1 * torch.randn_like(fake)

        disc_real = disc(real_noisy).reshape(-1)
        disc_fake = disc(fake_noisy).reshape(-1)
        real_labels = torch.full_like(disc_real, 0.9)
        fake_labels = torch.zeros_like(disc_fake)

        lossD_real = criterion(disc_real, real_labels)
        lossD_fake = criterion(disc_fake, fake_labels)
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake).reshape(-1)
        gen_labels = torch.ones_like(output)
        lossG = criterion(output, gen_labels)

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}] [Batch {batch_idx}] Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

    print(f" [Epoch {epoch+1}] Final Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    sample_path = f"{sample_dir}/epoch_{epoch+1}.png"
    with torch.no_grad():
        fake = gen(fixed_noise)
        save_image(fake * 0.5 + 0.5, sample_path)
    show_generated_image(sample_path, epoch + 1)

    if lossG.item() < best_lossG:
        best_lossG = lossG.item()
        torch.save(gen.state_dict(), f"{checkpoint_dir}/best_generator.pth")
        print(f" Best generator saved at epoch {epoch+1} with Loss G: {best_lossG:.4f}")

    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'gen_state_dict': gen.state_dict(),
            'disc_state_dict': disc.state_dict(),
            'opt_gen_state_dict': opt_gen.state_dict(),
            'opt_disc_state_dict': opt_disc.state_dict(),
        }, f"{checkpoint_dir}/dcgan_epoch_{epoch+1}.pth")


In [ ]:
import os
import torch
from torchvision.utils import save_image
import shutil
from pytorch_fid.fid_score import calculate_fid_given_paths
from glob import glob

# === Paths ===
checkpoint_dir = "/content/drive/MyDrive/dcgan_checkpoints"
sample_dir = "/content/fid_temp/generated"
real_dir = "/content/fid_temp/real"
os.makedirs(sample_dir, exist_ok=True)
os.makedirs(real_dir, exist_ok=True)

# === Settings ===
z_dim = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

# === Load Real Images (first time only) ===
from glob import glob
from PIL import Image
from torchvision import transforms

real_images = glob('./vggface2_processed/*/*.png')[:1000]
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
])

# Populate real_dir (only once)
if len(os.listdir(real_dir)) < 1000:
    for i, path in enumerate(real_images):
        img = Image.open(path).convert("RGB")
        img = transform(img)
        save_image(img, f"{real_dir}/real_{i}.png")

# === Evaluate FID for Models ===
fid_scores = {}

for epoch in range(145, 195):  # Update as needed
    try:
        ckpt_path = f"{checkpoint_dir}/dcgan_epoch_{epoch}.pth"
        print(f"\n Evaluating model: {ckpt_path}")

        # Load Generator
        checkpoint = torch.load(ckpt_path, map_location=device)
        gen.load_state_dict(checkpoint['gen_state_dict'])
        gen.eval()

        # Clean generated folder
        shutil.rmtree(sample_dir)
        os.makedirs(sample_dir, exist_ok=True)

        # Generate fake images
        with torch.no_grad():
            for i in range(1000):  # Use 1000 to match real samples
                noise = torch.randn(1, z_dim, 1, 1, device=device)
                fake_img = gen(noise).detach().cpu()
                save_image(fake_img * 0.5 + 0.5, f"{sample_dir}/fake_{i}.png")

        # Calculate FID
        fid = calculate_fid_given_paths([sample_dir, real_dir], batch_size=32, device=device, dims=2048)
        fid_scores[epoch] = fid
        print(f" FID for epoch {epoch}: {fid:.4f}")

    except Exception as e:
        print(f" Error at epoch {epoch}: {e}")

# === Show Best Result ===
if fid_scores:
    best_epoch = min(fid_scores, key=fid_scores.get)
    print("\n BEST MODEL:")
    print(f"Epoch {best_epoch} with FID: {fid_scores[best_epoch]:.4f}")

    # Save FID results
    with open("fid_results.txt", "w") as f:
        for epoch, fid in fid_scores.items():
            f.write(f"Epoch {epoch}: FID = {fid:.4f}\n")
        f.write(f"\nBest Epoch: {best_epoch} | FID = {fid_scores[best_epoch]:.4f}\n")
else:
    print(" No FID scores were computed.")



 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_145.pth


100%|██████████| 32/32 [00:00<00:00, 46.46it/s]


 FID for epoch 145: 83.0775

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_146.pth
 Error at epoch 146: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_146.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_147.pth
 Error at epoch 147: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_147.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_148.pth
 Error at epoch 148: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_148.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_149.pth
 Error at epoch 149: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_149.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_150.pth


100%|██████████| 32/32 [00:00<00:00, 40.06it/s]


 FID for epoch 150: 81.2912

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_151.pth
 Error at epoch 151: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_151.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_152.pth
 Error at epoch 152: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_152.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_153.pth
 Error at epoch 153: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_153.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_154.pth
 Error at epoch 154: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_154.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_155.pth


...

 FID for epoch 180: 83.9528

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_181.pth
 Error at epoch 181: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_181.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_182.pth
 Error at epoch 182: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_182.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_183.pth
 Error at epoch 183: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_183.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_184.pth
 Error at epoch 184: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_184.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_185.pth


100%|██████████| 32/32 [00:00<00:00, 44.58it/s]


 FID for epoch 185: 83.1409

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_186.pth
 Error at epoch 186: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_186.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_187.pth
 Error at epoch 187: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_187.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_188.pth
 Error at epoch 188: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_188.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_189.pth
 Error at epoch 189: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_189.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_190.pth


100%|██████████| 32/32 [00:00<00:00, 45.06it/s]


 FID for epoch 190: 78.5558

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_191.pth
 Error at epoch 191: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_191.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_192.pth
 Error at epoch 192: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_192.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_193.pth
 Error at epoch 193: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_193.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_194.pth
 Error at epoch 194: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_194.pth'

 BEST MODEL:
Epoch 190 with FID: 78.5558


In [ ]:
import os
import torch
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image

checkpoint_path = "/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_190.pth"
num_epochs = 110
z_dim = 100
best_lossG = float("inf")

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    gen.load_state_dict(checkpoint['gen_state_dict'])
    disc.load_state_dict(checkpoint['disc_state_dict'])
    opt_gen.load_state_dict(checkpoint['opt_gen_state_dict'])
    opt_disc.load_state_dict(checkpoint['opt_disc_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resumed from epoch {start_epoch}")
else:
    print("Checkpoint not found. Starting from scratch.")
    start_epoch = 0

def show_generated_image(image_path, epoch):
    img = Image.open(image_path)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Generated Samples - Epoch {epoch}")
    plt.show()

for epoch in range(start_epoch, start_epoch + num_epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(real.size(0), z_dim, 1, 1).to(device)
        fake = gen(noise)

        real_noisy = real + 0.1 * torch.randn_like(real)
        fake_noisy = fake.detach() + 0.1 * torch.randn_like(fake)

        disc_real = disc(real_noisy).reshape(-1)
        disc_fake = disc(fake_noisy).reshape(-1)
        real_labels = torch.full_like(disc_real, 0.9)
        fake_labels = torch.zeros_like(disc_fake)

        lossD_real = criterion(disc_real, real_labels)
        lossD_fake = criterion(disc_fake, fake_labels)
        lossD = (lossD_real + lossD_fake) / 2

        disc.zero_grad()
        lossD.backward()
        opt_disc.step()

        output = disc(fake).reshape(-1)
        gen_labels = torch.ones_like(output)
        lossG = criterion(output, gen_labels)

        gen.zero_grad()
        lossG.backward()
        opt_gen.step()

        if batch_idx % 100 == 0:
            print(f"[Epoch {epoch+1}] [Batch {batch_idx}] Loss D: {lossD.item():.4f}, Loss G: {lossG.item():.4f}")

    print(f" [Epoch {epoch+1}] Final Loss D: {lossD:.4f}, Loss G: {lossG:.4f}")

    sample_path = f"{sample_dir}/epoch_{epoch+1}.png"
    with torch.no_grad():
        fake = gen(fixed_noise)
        save_image(fake * 0.5 + 0.5, sample_path)
    show_generated_image(sample_path, epoch + 1)

    if lossG.item() < best_lossG:
        best_lossG = lossG.item()
        torch.save(gen.state_dict(), f"{checkpoint_dir}/best_generator.pth")
        print(f" Best generator saved at epoch {epoch+1} with Loss G: {best_lossG:.4f}")

    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'gen_state_dict': gen.state_dict(),
            'disc_state_dict': disc.state_dict(),
            'opt_gen_state_dict': opt_gen.state_dict(),
            'opt_disc_state_dict': opt_disc.state_dict(),
        }, f"{checkpoint_dir}/dcgan_epoch_{epoch+1}.pth")


In [ ]:
import os
import torch
from torchvision.utils import save_image
import shutil
from pytorch_fid.fid_score import calculate_fid_given_paths
from glob import glob

# === Paths ===
checkpoint_dir = "/content/drive/MyDrive/dcgan_checkpoints"
sample_dir = "/content/fid_temp/generated"
real_dir = "/content/fid_temp/real"
os.makedirs(sample_dir, exist_ok=True)
os.makedirs(real_dir, exist_ok=True)

# === Settings ===
z_dim = 100
device = "cuda" if torch.cuda.is_available() else "cpu"

# === Load Real Images (first time only) ===
from glob import glob
from PIL import Image
from torchvision import transforms

real_images = glob('./vggface2_processed/*/*.png')[:1000]
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
])

# Populate real_dir (only once)
if len(os.listdir(real_dir)) < 1000:
    for i, path in enumerate(real_images):
        img = Image.open(path).convert("RGB")
        img = transform(img)
        save_image(img, f"{real_dir}/real_{i}.png")

# === Evaluate FID for Models ===
fid_scores = {}

for epoch in range(190, 300):  # Update as needed
    try:
        ckpt_path = f"{checkpoint_dir}/dcgan_epoch_{epoch}.pth"
        print(f"\n Evaluating model: {ckpt_path}")

        # Load Generator
        checkpoint = torch.load(ckpt_path, map_location=device)
        gen.load_state_dict(checkpoint['gen_state_dict'])
        gen.eval()

        # Clean generated folder
        shutil.rmtree(sample_dir)
        os.makedirs(sample_dir, exist_ok=True)

        # Generate fake images
        with torch.no_grad():
            for i in range(1000):  # Use 1000 to match real samples
                noise = torch.randn(1, z_dim, 1, 1, device=device)
                fake_img = gen(noise).detach().cpu()
                save_image(fake_img * 0.5 + 0.5, f"{sample_dir}/fake_{i}.png")

        # Calculate FID
        fid = calculate_fid_given_paths([sample_dir, real_dir], batch_size=32, device=device, dims=2048)
        fid_scores[epoch] = fid
        print(f" FID for epoch {epoch}: {fid:.4f}")

    except Exception as e:
        print(f" Error at epoch {epoch}: {e}")

# === Show Best Result ===
if fid_scores:
    best_epoch = min(fid_scores, key=fid_scores.get)
    print("\n BEST MODEL:")
    print(f"Epoch {best_epoch} with FID: {fid_scores[best_epoch]:.4f}")

    # Save FID results
    with open("fid_results.txt", "w") as f:
        for epoch, fid in fid_scores.items():
            f.write(f"Epoch {epoch}: FID = {fid:.4f}\n")
        f.write(f"\nBest Epoch: {best_epoch} | FID = {fid_scores[best_epoch]:.4f}\n")
else:
    print(" No FID scores were computed.")



 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_190.pth


100%|██████████| 32/32 [00:00<00:00, 46.28it/s]


 FID for epoch 190: 79.1998

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_191.pth
 Error at epoch 191: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_191.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_192.pth
 Error at epoch 192: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_192.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_193.pth
 Error at epoch 193: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_193.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_194.pth
 Error at epoch 194: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_194.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_195.pth


100%|██████████| 32/32 [00:00<00:00, 45.75it/s]


 FID for epoch 195: 85.7797

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_196.pth
 Error at epoch 196: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_196.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_197.pth
 Error at epoch 197: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_197.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_198.pth
 Error at epoch 198: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_198.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_199.pth
 Error at epoch 199: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_199.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_200.pth


...

 FID for epoch 285: 79.9357

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_286.pth
 Error at epoch 286: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_286.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_287.pth
 Error at epoch 287: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_287.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_288.pth
 Error at epoch 288: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_288.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_289.pth
 Error at epoch 289: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_289.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_290.pth


100%|██████████| 32/32 [00:00<00:00, 46.48it/s]


 FID for epoch 290: 76.5001

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_291.pth
 Error at epoch 291: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_291.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_292.pth
 Error at epoch 292: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_292.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_293.pth
 Error at epoch 293: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_293.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_294.pth
 Error at epoch 294: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_294.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_295.pth


100%|██████████| 32/32 [00:00<00:00, 47.20it/s]


 FID for epoch 295: 76.8271

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_296.pth
 Error at epoch 296: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_296.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_297.pth
 Error at epoch 297: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_297.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_298.pth
 Error at epoch 298: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_298.pth'

 Evaluating model: /content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_299.pth
 Error at epoch 299: [Errno 2] No such file or directory: '/content/drive/MyDrive/dcgan_checkpoints/dcgan_epoch_299.pth'

 BEST MODEL:
Epoch 280 with FID: 75.8781
